# 01 - Limpeza de Dados
Autora: Fernanda Baptista de Siqueira  
Curso: MBA em Tecnologia para Negócios – AI, Data Science e Big Data  
Tema: Análise de Acidentes de Trânsito em Porto Alegre (2020–2024)  
Origem DataFrame: Equipe Armazém de Dados de Mobilidade - EAMOB/CIET  
https://dadosabertos.poa.br/dataset/acidentes-de-transito-acidentes (11/05/2025)  

### 1. Importação de Bibliotecas e Funções

In [6]:
# Imports principais
import pandas as pd
import numpy as np

# Imports do projeto
from utils import (
    PATH_RAW, PATH_CLEAN,
    COLS_VEICULOS, resumo_df,
    salvar_parquet, checar_nulos,
    ajustar_tipos, criar_feat_temp
)

### 2. Carga de Dados e Leitura Inicial

In [2]:
arquivo = PATH_RAW + 'acidentes.csv'

try:
    df = pd.read_csv(
        arquivo,
        sep=';',
        encoding='utf-8',
        low_memory=False,
        dtype=str
        )
    print(f"CSV original lido com sucesso!")
except FileNotFoundError:
    print(f"Erro: Arquivo '{arquivo}' não encontrado. Verifique o caminho.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

# Exibe um resumo inicial
print("--- Resumo do DataFrame Limpo e Preparado ---")
resumo_df(df)

# Checa a quantidade de nulos por coluna
print("\n--- Verificação de Nulos ---")
checar_nulos(df)

CSV original lido com sucesso!
--- Resumo do DataFrame Limpo e Preparado ---
Dimensões: (69521, 34)

Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69521 entries, 0 to 69520
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   data_extracao  69521 non-null  object
 1   predial1       65442 non-null  object
 2   queda_arr      69521 non-null  object
 3   data           69521 non-null  object
 4   feridos        69521 non-null  object
 5   feridos_gr     69521 non-null  object
 6   mortes         69521 non-null  object
 7   morte_post     69521 non-null  object
 8   fatais         69521 non-null  object
 9   auto           69521 non-null  object
 10  taxi           69521 non-null  object
 11  lotacao        69521 non-null  object
 12  onibus_urb     69521 non-null  object
 13  onibus_met     69521 non-null  object
 14  onibus_int     69521 non-null  object
 15  caminhao       69521 non-null  object
 16  moto

,data_extracao,predial1,queda_arr,data,feridos,feridos_gr,mortes,morte_post,fatais,auto,taxi,lotacao,onibus_urb,onibus_met,onibus_int,caminhao,moto,carroca,bicicleta,outro,cont_vit,ups,patinete,idacidente,longitude,latitude,log1,log2,tipo_acid,dia_sem,hora,noite_dia,regiao,consorcio
0,2025-06-01 01:33:13,0,0.0,2020-10-17 00:00:00,1,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,1,5,0,190816,0,0,R MARCOS MOREIRA,R GASTON ENGLERT,ABALROAMENTO,SÁBADO,19:00:00.0000000,NOITE,NORTE,NaN
1,2025-06-01 01:33:13,598,0.0,2020-01-01 00:00:00,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,5,0,669089,NaN,NaN,AV BENTO GONCALVES,NaN,ABALROAMENTO,QUARTA-FEIRA,03:00:00.0000000,NOITE,LESTE,NaN
2,2025-06-01 01:33:13,1271,0.0,2020-01-01 00:00:00,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,5,0,669097,NaN,NaN,AV INDEPENDENCIA,NaN,ATROPELAMENTO,QUARTA-FEIRA,23:00:00.0000000,NOITE,LESTE,NaN
3,2025-06-01 01:33:13,1901,0.0,2020-01-02 00:00:00,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,5,0,669098,NaN,NaN,AV EDUARDO PRADO,NaN,ATROPELAMENTO,QUINTA-FEIRA,00:05:00.0000000,NOITE,SUL,NaN
4,2025-06-01 01:33:13,3302,0.0,2020-01-02 00:00:00,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,5,0,669099,-51.21153485762743,-30.081535213015123,AV TERESOPOLIS,NaN,ABALROAMENTO,QUINTA-FEIRA,09:00:00.0000000,DIA,SUL,NaN



--- Verificação de Nulos ---


consorcio       96.83
log2            72.42
latitude        14.82
longitude       14.82
predial1         5.87
hora             0.80
log1             0.07
data_extracao    0.00
queda_arr        0.00
data             0.00
taxi             0.00
lotacao          0.00
feridos          0.00
feridos_gr       0.00
mortes           0.00
morte_post       0.00
fatais           0.00
auto             0.00
carroca          0.00
moto             0.00
caminhao         0.00
onibus_int       0.00
onibus_met       0.00
onibus_urb       0.00
bicicleta        0.00
outro            0.00
patinete         0.00
idacidente       0.00
cont_vit         0.00
ups              0.00
dia_sem          0.00
tipo_acid        0.00
noite_dia        0.00
regiao           0.00
dtype: float64

### 3. Limpeza e Tratamento Inicial  
#### 3.1. Padronização e Remoção de Colunas

In [4]:
# Remove espaços, converte para minúsculas e substitui espaços por '_' (snake_case)
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

# Remove colunas que não serão utilizadas na análise ou modelagem
df = df.drop(columns=[
    'data_extracao', 'consorcio', 'latitude',
    'longitude', 'mortes', 'morte_post'
    ], 
    errors='ignore')

# Lista de colunas restantes
print("Colunas após a limpeza inicial:")
print(df.columns)

Colunas após a limpeza inicial:
Index(['predial1', 'queda_arr', 'data', 'feridos', 'feridos_gr', 'fatais',
       'auto', 'taxi', 'lotacao', 'onibus_urb', 'onibus_met', 'onibus_int',
       'caminhao', 'moto', 'carroca', 'bicicleta', 'outro', 'cont_vit', 'ups',
       'patinete', 'idacidente', 'log1', 'log2', 'tipo_acid', 'dia_sem',
       'hora', 'noite_dia', 'regiao'],
      dtype='object')


#### 3.2. Tratamento de Nulos/Inválidos

In [7]:
# Converte 'data' para datetime, tratando erros
df['data'] = pd.to_datetime(df['data'], errors='coerce')

# Converte colunas numéricas que deveriam ser strings para evitar erros
df['predial1'] = pd.to_numeric(df['predial1'], errors='coerce').astype('Int64')
df['log2'] = df['log2'].replace(np.nan, '').astype(str).str.strip()
df.loc[df['log2'] == '', 'log2'] = np.nan


# LÓGICA: Remove registros de esquina (predial1 é 0 ou nulo) que não possuem log2 (rua 2)
antes = len(df)
condicao_esquina_incompleta = (df['predial1'].isnull() | (df['predial1'] == 0)) & (df['log2'].isnull())
df = df[~condicao_esquina_incompleta]
print(f"{antes - len(df)} registros de esquina sem 'log2' foram removidos.")

# Remove linhas com valores nulos em colunas essenciais
df = df.dropna(subset=['data', 'hora', 'log1', 'regiao'])

# Filtra o DataFrame para o período de análise
df = df[(df['data'] >= '2020-01-01') & (df['data'] <= '2025-04-01')]
print("Registros fora do escopo (2020-2025) removidos.")

# Remove chaves duplicadas
df = df.drop_duplicates(subset='idacidente')

# Padroniza dias da semana
df["dia_sem"] = (
    df["dia_sem"]
    .str.title()
    .str.replace("-Feira", "", regex=False)
)

print(f"\nDataFrame após remoção de inválidos. Dimensões: {df.shape}")

3017 registros de esquina sem 'log2' foram removidos.
Registros fora do escopo (2020-2025) removidos.

DataFrame após remoção de inválidos. Dimensões: (65866, 28)


### 4. Conversão de Tipos e Preparação

In [9]:
# Converte a coluna 'hora' para o formato timedelta
df['hora'] = pd.to_timedelta(df['hora'], errors='coerce')

# Chama a nova função unificada para ajustar todos os tipos de uma só vez
df = ajustar_tipos(df)

print("Tipos de dados convertidos e preparados para EDA e modelagem.")
df.info()

Tipos de dados convertidos e preparados para EDA e modelagem.
<class 'pandas.core.frame.DataFrame'>
Index: 65866 entries, 0 to 69520
Data columns (total 28 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   predial1    63669 non-null  float64        
 1   queda_arr   65866 non-null  float64        
 2   data        65866 non-null  datetime64[ns] 
 3   feridos     65866 non-null  float64        
 4   feridos_gr  65866 non-null  float64        
 5   fatais      65866 non-null  float64        
 6   auto        65866 non-null  float64        
 7   taxi        65866 non-null  float64        
 8   lotacao     65866 non-null  float64        
 9   onibus_urb  65866 non-null  float64        
 10  onibus_met  65866 non-null  float64        
 11  onibus_int  65866 non-null  float64        
 12  caminhao    65866 non-null  float64        
 13  moto        65866 non-null  float64        
 14  carroca     65866 non-null  float64        
 

### 5. Engenharia de Atributos (Feature Engineering)

In [11]:
# Cria coluna 'data_hora' completa
df['data_hora'] = df['data'] + df['hora']

# Usa a função de utils para criar features de ano, mês, dia e hora
df = criar_feat_temp(df, 'data_hora')

# Cria coluna com o total de vítimas
df["total_vitimas"] = (
    df["feridos"].fillna(0) +
    df["fatais"].fillna(0)
)

# Cria coluna com a soma total de veículos
df['soma_veiculos'] = df[COLS_VEICULOS].sum(axis=1)

print("Colunas derivadas criadas com sucesso.")

Colunas derivadas criadas com sucesso.


### 6. Verificação Final

In [12]:
# Exibe um resumo final do DataFrame
print("--- Resumo do DataFrame Limpo e Preparado ---")
resumo_df(df)

# Checa a quantidade de nulos por coluna
print("\n--- Verificação Final de Nulos ---")
checar_nulos(df)

--- Resumo do DataFrame Limpo e Preparado ---
Dimensões: (65866, 35)

Info: 
<class 'pandas.core.frame.DataFrame'>
Index: 65866 entries, 0 to 69520
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   predial1       63669 non-null  float64        
 1   queda_arr      65866 non-null  float64        
 2   data           65866 non-null  datetime64[ns] 
 3   feridos        65866 non-null  float64        
 4   feridos_gr     65866 non-null  float64        
 5   fatais         65866 non-null  float64        
 6   auto           65866 non-null  float64        
 7   taxi           65866 non-null  float64        
 8   lotacao        65866 non-null  float64        
 9   onibus_urb     65866 non-null  float64        
 10  onibus_met     65866 non-null  float64        
 11  onibus_int     65866 non-null  float64        
 12  caminhao       65866 non-null  float64        
 13  moto           65866 non-null  flo

,predial1,queda_arr,data,feridos,feridos_gr,fatais,auto,taxi,lotacao,onibus_urb,onibus_met,onibus_int,caminhao,moto,carroca,bicicleta,outro,cont_vit,ups,patinete,idacidente,log1,log2,tipo_acid,dia_sem,hora,noite_dia,regiao,data_hora,ano,mes,dia,hora_int,total_vitimas,soma_veiculos
0,0.00,0.00,2020-10-17,1.00,0.00,0.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,5.00,0.00,"190,816.00",R MARCOS MOREIRA,R GASTON ENGLERT,ABALROAMENTO,Sábado,0 days 19:00:00,NOITE,NORTE,2020-10-17 19:00:00,2020,10,17,19,1.00,4.00
1,598.00,0.00,2020-01-01,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,5.00,0.00,"669,089.00",AV BENTO GONCALVES,<NA>,ABALROAMENTO,Quarta,0 days 03:00:00,NOITE,LESTE,2020-01-01 03:00:00,2020,1,1,3,1.00,2.00
2,"1,271.00",0.00,2020-01-01,1.00,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,5.00,0.00,"669,097.00",AV INDEPENDENCIA,<NA>,ATROPELAMENTO,Quarta,0 days 23:00:00,NOITE,LESTE,2020-01-01 23:00:00,2020,1,1,23,1.00,1.00
3,"1,901.00",0.00,2020-01-02,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,5.00,0.00,"669,098.00",AV EDUARDO PRADO,<NA>,ATROPELAMENTO,Quinta,0 days 00:05:00,NOITE,SUL,2020-01-02 00:05:00,2020,1,2,0,2.00,1.00
4,"3,302.00",0.00,2020-01-02,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,5.00,0.00,"669,099.00",AV TERESOPOLIS,<NA>,ABALROAMENTO,Quinta,0 days 09:00:00,DIA,SUL,2020-01-02 09:00:00,2020,1,2,9,1.00,2.00



--- Verificação Final de Nulos ---


log2            71.19
predial1         3.34
queda_arr        0.00
data             0.00
feridos          0.00
fatais           0.00
feridos_gr       0.00
taxi             0.00
lotacao          0.00
onibus_urb       0.00
auto             0.00
onibus_int       0.00
caminhao         0.00
moto             0.00
carroca          0.00
bicicleta        0.00
outro            0.00
cont_vit         0.00
onibus_met       0.00
ups              0.00
patinete         0.00
idacidente       0.00
log1             0.00
tipo_acid        0.00
dia_sem          0.00
hora             0.00
noite_dia        0.00
regiao           0.00
data_hora        0.00
ano              0.00
mes              0.00
dia              0.00
hora_int         0.00
total_vitimas    0.00
soma_veiculos    0.00
dtype: float64

### 7. Salvamento DataFrame Tratado

In [13]:
# Salva dataframes tratado
salvar_parquet(df, PATH_CLEAN + "df_limpo.parquet")

Salvo: ../dados/intermediarios/df_limpo.parquet


-----------------------------------

### Referências Essenciais para Preparação de Dados

* Card, S.K. Card, J. Mackinlay, B. Shneiderman.*Readings in Information Visualization: Using Vision to Think*. 1999. 

* Dasu, T.; Johnson, T. *Exploratory Data Mining and Data Cleaning*. Wiley-IEEE. 2003.

* Feyyad, U. M. *Data mining and knowledge discovery: making sense out of data*. IEEE Expert. 1996.

* Han, Jiawei Han, Micheline Kamber, Jian Pei. *Data Mining: Concepts and Techniques*. 3rd Edition, Morgan Kaufmann, 2011.

* Milani, A. M. P. Milani, F. V. Paulovich, I. H. Manssoour. *Visualization in the preprocessing phase: Getting insights from enterprise professionals*. Information Visualization. 2020.

* PROVOST, F.; FAWCETT, T. *Data Science para negócios: O que você precisa saber sobre mineração de dados e pensamento analítico de dados*. Alta Books, 2016.

* Shearer, Colin Shearer. *The CRISP-DM Model: The New Blueprint for Data Mining*. Journal of Data Warehousing. 2000.

* TUFTE, Edward R. *Envisioning Information*. 1990.

* VANDERPLAS, Jake. *Python data science handbook: Essential tools for working with data*. 2017.

* Wickham, H. *Tidy Data*. Journal of Statistical Software. 2014. 